In [4]:
import sys
from pathlib import Path

# Detect project root automatically relative to notebook location
project_root = Path.cwd().resolve().parents[0]  # assumes notebook in /notebooks/
if (project_root / "src").exists() and str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.tngmw.config import RAW, PLOTS, SIMULATION, SNAPNUM, SUBID

In [15]:
import yt
import trident
from src.tngmw.config import RAW, PLOTS

# Load the dataset
ds = yt.load(RAW / "subhalo_432106_cutout_full.hdf5")
print("Fields available after load:", ds.field_list)

# Add ion fields
ions = ["H I", "C IV", "O VI", "Na I", "Ca II"]
trident.add_ion_fields(ds, ions=ions)

# Print fields again to verify what got added
print("Fields after ion addition:", ds.field_list)
print("Derived Fields:", ds.derived_field_list)



yt : [INFO     ] 2025-11-08 23:09:22,616 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2025-11-08 23:09:22,638 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-11-08 23:09:22,639 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-11-08 23:09:22,639 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-11-08 23:09:22,640 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2025-11-08 23:09:22,640 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-11-08 23:09:22,640 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-11-08 23:09:22,641 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-11-08 23:09:22,641 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-11-08 23:09:22,642 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-11-08 23:09:22,643 Parameters: hubble_con

Fields available after load: [('PartType0', 'CenterOfMass'), ('PartType0', 'Coordinates'), ('PartType0', 'Density'), ('PartType0', 'ElectronAbundance'), ('PartType0', 'EnergyDissipation'), ('PartType0', 'GFM_AGNRadiation'), ('PartType0', 'GFM_CoolingRate'), ('PartType0', 'GFM_Metallicity'), ('PartType0', 'GFM_MetalsTagged'), ('PartType0', 'GFM_Metals_00'), ('PartType0', 'GFM_Metals_01'), ('PartType0', 'GFM_Metals_02'), ('PartType0', 'GFM_Metals_03'), ('PartType0', 'GFM_Metals_04'), ('PartType0', 'GFM_Metals_05'), ('PartType0', 'GFM_Metals_06'), ('PartType0', 'GFM_Metals_07'), ('PartType0', 'GFM_Metals_08'), ('PartType0', 'GFM_Metals_09'), ('PartType0', 'GFM_WindDMVelDisp'), ('PartType0', 'GFM_WindHostHaloMass'), ('PartType0', 'InternalEnergy'), ('PartType0', 'Machnumber'), ('PartType0', 'MagneticField'), ('PartType0', 'MagneticFieldDivergence'), ('PartType0', 'Masses'), ('PartType0', 'NeutralHydrogenAbundance'), ('PartType0', 'ParticleIDs'), ('PartType0', 'Potential'), ('PartType0', 'S

In [33]:
import requests
from src.tngmw.config import SIMULATION, SNAPNUM, SUBID, TNG_API_KEY

url = f"https://www.tng-project.org/api/{SIMULATION}/snapshots/{SNAPNUM}/subhalos/{SUBID}/"
resp = requests.get(url, headers={"api-key": TNG_API_KEY})
resp.raise_for_status()
detail = resp.json()

pos_x = detail["pos_x"]
pos_y = detail["pos_y"]
pos_z = detail["pos_z"]

print("Subhalo position (ckpc/h):", pos_x, pos_y, pos_z)

Subhalo position (ckpc/h): 7892.83 8587.43 2216.5


In [42]:
def make_ion_proj(ds, center, width_kpc=600):
    ions = ["H I", "C IV", "O VI", "Na I", "Ca II"]
    trident.add_ion_fields(ds, ions=ions)
    fields = {
        "H I": ("gas", "H_p0_number_density"),
        "C IV": ("gas", "C_p3_number_density"),
        "O VI": ("gas", "O_p5_number_density"),
        "Na I": ("gas", "Na_p0_number_density"),
        "Ca II": ("gas", "Ca_p1_number_density"),
    }

    for name, fld in fields.items():
        if fld in ds.derived_field_list:
            p = yt.ProjectionPlot(ds, "z", fld, center=center, width=(width_kpc, "kpc"))
            p.set_cmap(fld, "viridis")
            p.save(PLOTS / f"{name.replace(' ', '_')}.png")

In [43]:
make_ion_proj(ds, center=[pos_x, pos_y, pos_z], width_kpc=300)

yt : [WARNING  ] 2025-11-08 23:28:35,134 Field ('gas', 'H_p0_ion_fraction') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,137 Field ('gas', 'H_p0_number_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,137 Field ('gas', 'H_p0_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,138 Field ('gas', 'H_p0_mass') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,138 Field ('gas', 'C_p3_ion_fraction') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,139 Field ('gas', 'C_p3_number_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,139 Field ('gas', 'C_p3_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,139 Field ('gas', 'C_p3_mass') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,140 Field ('gas', 'O_p5_ion_fraction') already exists. Not clobbering.
yt : [WARNING  ] 2025-11-08 23:28:35,140 Field ('g

In [59]:
import yt
from src.tngmw.config import RAW, PLOTS

ds = yt.load(RAW / "subhalo_432106_cutout_full.hdf5")
center = [pos_x, pos_y, pos_z]          # from API
width  = (700, "kpc")

axis  = "z"
vfield = ("gas", f"velocity_{axis}")    # ('gas','velocity_z')

p = yt.ProjectionPlot(
    ds, axis, vfield,
    center=center, width=width,
    weight_field=("gas","density"), # density-weighted mean LOS velocity
    buff_size=(2048,2048),
)
p.set_unit(vfield, "km/s")
p.set_cmap(vfield, "RdBu_r")            # diverging for ± velocities
p.set_zlim(vfield, -60, 150)           # adjust to your system
out = PLOTS / f"los_velocity_{axis}_{int(width[0])}kpc.png"
p.save(out)
print(out)

yt : [INFO     ] 2025-11-08 23:54:18,142 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2025-11-08 23:54:18,182 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-11-08 23:54:18,182 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-11-08 23:54:18,183 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-11-08 23:54:18,183 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2025-11-08 23:54:18,183 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-11-08 23:54:18,184 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-11-08 23:54:18,184 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-11-08 23:54:18,184 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-11-08 23:54:18,184 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-11-08 23:54:18,184 Parameters: hubble_con

/Users/wavefunction/ASU Dropbox/Tanmay Singh/TNG-MW/plots/los_velocity_z_700kpc.png
